# Load Required Packages

In [ ]:
import pandas as pd
import numpy as np


# Load Dummy Data

In [ ]:
np.random.seed(42)

# Generate vendors data
vendors = pd.DataFrame({
    'vendor_id': [f'V{i}' for i in range(1, 201)],
    'vendor_country': np.random.choice(['Country A', 'Country B', 'Country C'], size=200)
})

# Generate items data
items = pd.DataFrame({
    'item_id': range(1, 1001),
    'PP_Type': np.random.choice(['Type1', 'Type2', 'Type3', 'Type4'], size=1000)
})

# Merge items with vendors to create a merged dataframe
merged_df = items.copy()
merged_df['vendor_id'] = np.random.choice(vendors['vendor_id'], size=1000)
merged_df['vendor_country'] = merged_df['vendor_id'].map(vendors.set_index('vendor_id')['vendor_country'])
merged_df['cost_per_unit'] = np.random.uniform(10, 100, size=1000)

# Generate factory demand data
factory_demand_df = pd.DataFrame({
    'factory_id': [f'F{i}' for i in range(1, 501)],
    'factory_country': np.random.choice(['Country A', 'Country B', 'Country C'], size=500),
    'VOLUME': np.random.randint(50, 200, size=500)
})

# Generate target volume data for vendors
target_volume_df = pd.DataFrame({
    'vendor_id': [f'V{i}' for i in range(1, 201)],
    'target_allocation_pct': np.random.uniform(0.05, 0.15, size=200),
    'acceptable_range_pct': np.random.uniform(5, 10, size=200),
    'PP_Type': np.random.choice(['Type1', 'Type2', 'Type3', 'Type4'], size=200)
})

# Define constraints
constraints = {
    'unacceptable_pairs': [(np.random.randint(1, 1001), f'F{np.random.randint(1, 501)}') for _ in range(10)]
}


# Define Functions

## Pre-Processing

In [ ]:
def preprocess_data(merged_df, constraints=None):
    if constraints and 'unacceptable_pairs' in constraints:
        for item_id, factory_id in constraints['unacceptable_pairs']:
            merged_df = merged_df[~((merged_df['item_id'] == item_id) & (merged_df['factory_id'] == factory_id))]
    return merged_df


## Winning Supplier

In [ ]:
def select_winning_supplier(merged_df, factory_demand_df):
    selected_suppliers = []

    for _, factory in factory_demand_df.iterrows():
        factory_id = factory['factory_id']
        factory_country = factory['factory_country']

        # Prioritize local suppliers first
        local_suppliers = merged_df[merged_df['vendor_country'] == factory_country]
        non_local_suppliers = merged_df[merged_df['vendor_country'] != factory_country]

        for suppliers in [local_suppliers, non_local_suppliers]:
            sorted_suppliers = suppliers.sort_values(by='cost_per_unit')
            for _, item in sorted_suppliers.iterrows():
                selected_suppliers.append({
                    'item_id': item['item_id'],
                    'vendor_id': item['vendor_id'],
                    'factory_id': factory_id,
                    'cost_per_unit': item['cost_per_unit'],
                    'vendor_country': item['vendor_country'],
                    'local_for_local': item['vendor_country'] == factory_country,
                    # Include other relevant item and vendor details as needed
                })

    return pd.DataFrame(selected_suppliers)


## Winning Factory

In [ ]:
def allocate_to_factories(selected_suppliers, target_volume_df, factory_demand_df):
    allocation_result = []

    for _, allocation in selected_suppliers.iterrows():
        vendor_id = allocation['vendor_id']
        item_id = allocation['item_id']
        factory_id = allocation['factory_id']

        target_volume_data = target_volume_df[target_volume_df['vendor_id'] == vendor_id]
        factory_data = factory_demand_df[factory_demand_df['factory_id'] == factory_id]

        if not target_volume_data.empty and not factory_data.empty:
            target_allocation = target_volume_data['target_allocation_pct'].iloc[0] * factory_data['VOLUME'].iloc[0]

            allocation_result.append({
                'item_id': item_id,
                'factory_id': factory_id,
                'vendor_id': vendor_id,
                'allocated_units': target_allocation,
                # Carry forward additional details as necessary
                'cost_per_unit': allocation['cost_per_unit'],
                'vendor_country': allocation['vendor_country'],
                'local_for_local': allocation['local_for_local'],
                # Add other relevant details here
            })

    return pd.DataFrame(allocation_result)


# Execute the Workflow

In [ ]:
processed_merged_df = preprocess_data(merged_df, constraints)
winning_suppliers = select_winning_supplier(processed_merged_df, factory_demand_df)
allocation_results = allocate_to_factories(winning_suppliers, target_volume_df, factory_demand_df)


# Explore Results

In [ ]:
print("Basic Statistics:")
print(allocation_results.describe())


In [ ]:
print("\nAllocation Distribution by Factory:")
factory_allocation = allocation_results.groupby('factory_id')['allocated_units'].sum()
print(factory_allocation)


In [ ]:
print("\nAllocation Distribution by Vendor:")
vendor_allocation = allocation_results.groupby('vendor_id')['allocated_units'].sum()
print(vendor_allocation)


In [ ]:
allocation_results['total_cost'] = allocation_results['allocated_units'] * allocation_results['cost_per_unit']
print("\nTotal Cost per Factory:")
total_cost_per_factory = allocation_results.groupby('factory_id')['total_cost'].sum()
print(total_cost_per_factory)


In [ ]:
print("\nLocal-for-Local Allocations:")
local_allocations = allocation_results['local_for_local'].value_counts()
print(local_allocations)


In [ ]:
print("\nDetailed Allocation per Item:")
item_allocation = allocation_results.groupby(['item_id', 'factory_id', 'vendor_id']).agg({'allocated_units': 'sum', 'total_cost': 'sum'})
print(item_allocation)


In [ ]:
# Assuming PP_Type is part of the allocation results
print("\nAllocation Distribution by PP_Type:")
pp_type_allocation = allocation_results.groupby('PP_Type')['allocated_units'].sum()
print(pp_type_allocation)
